# Automação Web e Busca de Informações com Python

#### Desafio: 

O trabalho é de uma importadora que compra e vende commodities:
- Soja, Milho, Trigo, Petróleo, etc.

É preciso pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do preço ideal de compra. Se tiver, é preciso marcar como uma ação de compra para a equipe de operações.

Obs:

- Usar o selenium
- Importante: baixar o webdriver

In [102]:
import pandas as pd

tabela = pd.read_excel('commodites.xlsx')
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [103]:
from selenium import webdriver
#from unicodedata import normalize 
navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')
#send_keys('escreva algo') -> digita algo nele
#click() -> clica nele
#get_attribute() -> pega o valor dele

for linha in tabela.index:
    
    produto = tabela.loc[linha, 'Produto']
    link = f'https://www.melhorcambio.com/{produto}-hoje'
    link= link.replace("ó","o").replace("ã","a").replace("ç","c").replace("ú","u").replace("é","e").replace("á","a")
    #link = normalize("NFKD", link).encode("ascii", "ignore")
    #o ideal seria usar esse código, mas como são poucos caracteres, resolvir 'fazer na unha'
    navegador.get(link)
    preco_atual = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
    preco_atual = preco_atual.replace(".", "").replace(",",".")
    preco_atual = float(preco_atual)
    tabela.loc[linha, 'Preço Atual'] = preco_atual
    print(produto)
    print(preco_atual)   

Milho
57.43
Soja
133.82
Boi
267.75
Petróleo
377.68
Algodão
390.52
Açúcar
148.92
Café
1042.37
Ouro
313.1
Trigo
0.0
Tilápia
9.64


In [104]:
tabela['Comprar'] = tabela['Preço Ideal'] > tabela['Preço Atual']
display(tabela)

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,57.43,True
1,Soja,163.59,133.82,True
2,Boi,282.20,267.75,True
3,Petróleo,424.37,377.68,True
4,Algodão,497.76,390.52,True
5,Açúcar,136.23,148.92,False
6,Café,1092.87,1042.37,True
7,Ouro,321.77,313.10,True
8,Trigo,1549.11,0.00,True
9,Tilápia,9.05,9.64,False


In [105]:
navegador.quit() 
tabela.to_excel('commodites-atualizado.xlsx', index=False)